In [1]:
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Conv2D, MaxPooling2D, Dense, Flatten, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

2024-06-30 17:46:55.630509: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-30 17:46:55.630658: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-30 17:46:55.763038: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def armazenar_nomes_arquivos(caminho):
    nomes_arquivos = []
    for filename in os.listdir(caminho):
        filepath = os.path.join(caminho, filename)
        nomes_arquivos.append(filepath)

    return nomes_arquivos

def criar_df_classe_unica(nomes_arquivos, label):
    labels = []
    for i in range(len(nomes_arquivos)):
        labels.append(label)

    df_classe_unica = pd.DataFrame({'nomes_arquivos': nomes_arquivos, 'labels': labels})

    return df_classe_unica

def rotacionar_imagem(imagem):
    imagem_cinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)

    _, imagem_bin = cv2.threshold(imagem_cinza, 127, 255, cv2.THRESH_BINARY)

    contornos, _ = cv2.findContours(imagem_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    maior_contorno = max(contornos, key=cv2.contourArea)
    
    rect = cv2.minAreaRect(maior_contorno)
    box = cv2.boxPoints(rect)
    box = np.intp(box)

    _, dimensoes, angulo = rect
    
    largura, altura = dimensoes
    if largura < altura:
        angulo = angulo + 90 

    altura, largura = imagem.shape[:2]
    centro = (largura // 2, altura // 2)
    matriz_rotacao = cv2.getRotationMatrix2D(centro, angulo, 1.0)
    imagem_rotacionada = cv2.warpAffine(imagem, matriz_rotacao, (largura, altura))
    
    return imagem_rotacionada

def criar_modelo():
    inputr = Input(shape=(90000,), name='red')  
    inputg = Input(shape=(90000,), name='green')  
    inputb = Input(shape=(90000,), name='blue')  

    x1 = Dense(64, activation='relu')(inputr)
    x1 = Dense(32, activation='relu')(x1)

    x2 = Dense(64, activation='relu')(inputg)
    x2 = Dense(32, activation='relu')(x2)

    x3 = Dense(64, activation='relu')(inputb)
    x3 = Dense(32, activation='relu')(x3)

    combined = concatenate([x1, x2, x3])

    z = Dense(64, activation='relu')(combined)
    z = Dense(32, activation='relu')(z)
    output = Dense(1, activation='sigmoid')(z) 

    model = Model(inputs=[inputr, inputg, inputb], outputs=output)
    
    return model

# Importar imagens

In [3]:
kirmizi_path = '/kaggle/input/pistachio-image-dataset/Pistachio_Image_Dataset/Pistachio_Image_Dataset/Kirmizi_Pistachio'
siirt_path = '/kaggle/input/pistachio-image-dataset/Pistachio_Image_Dataset/Pistachio_Image_Dataset/Siirt_Pistachio'

kirmizi_arquivos = armazenar_nomes_arquivos(kirmizi_path)
df_kirmizi = criar_df_classe_unica(kirmizi_arquivos, 'kirmizi')

siirt_arquivos = armazenar_nomes_arquivos(siirt_path)
df_siirt = criar_df_classe_unica(siirt_arquivos, 'siirt')

df = pd.concat([df_kirmizi, df_siirt], axis=0, ignore_index=True)


##Label = 1 -> Kirmizi e label = 0 -> Siirt
df['labels'] = df['labels'].map({'kirmizi': 1, 'siirt': 0})

X = df['nomes_arquivos']
y = df['labels']

# Faz resize das imagens

In [4]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.5, stratify=y, random_state=42)

red_treino = []
green_treino = []
blue_treino = []

for arquivo_treino in X_treino:
    imagem_treino = cv2.imread(arquivo_treino)
    nl, nc, num_canais = imagem_treino.shape
    imagem_resize = cv2.resize(imagem_treino, None, fx=0.5, fy=0.5)
    imagem_resize = imagem_resize/255
    imagem_reshape = imagem_resize.reshape(imagem_resize.shape[0] * imagem_resize.shape[1], num_canais)
    
    red_treino.append(imagem_reshape[:, 0]) 
    green_treino.append(imagem_reshape[:, 1])
    blue_treino.append(imagem_reshape[:, 2])

red_treino = np.array(red_treino)
green_treino = np.array(green_treino)
blue_treino = np.array(blue_treino)


red_teste = []
green_teste = []
blue_teste = []

for arquivo_teste in X_teste:
    imagem_teste = cv2.imread(arquivo_teste)
    nl, nc, num_canais = imagem_teste.shape
    imagem_resize = cv2.resize(imagem_teste, None, fx=0.5, fy=0.5)
    imagem_resize = imagem_resize/255
    imagem_reshape = imagem_resize.reshape(imagem_resize.shape[0] * imagem_resize.shape[1], num_canais)
    
    red_teste.append(imagem_reshape[:, 0]) 
    green_teste.append(imagem_reshape[:, 1])
    blue_teste.append(imagem_reshape[:, 2])

red_teste = np.array(red_teste)
green_teste = np.array(green_teste)
blue_teste = np.array(blue_teste)

print(red_treino.shape)

(1074, 90000)


# Rotaciona Imagens

In [5]:
imagens_rotacionadas = []

for arquivo in X:
    imagem = cv2.imread(arquivo)
    nl, nc, num_canais = imagem.shape
    imagem_resize = cv2.resize(imagem, None, fx=0.5, fy=0.5)
    imagem_rotacionada = rotacionar_imagem(imagem_resize)
    imagens_rotacionadas.append(imagem_rotacionada)

imagens_rotacionadas = np.array(imagens_rotacionadas)

X_alinha_treino, X_alinha_teste, y_alinha_treino, y_alinha_teste = train_test_split(imagens_rotacionadas, y, test_size=0.5, stratify=y, random_state=42)

X_alinha_treino = X_alinha_treino.reshape(-1, 90000, 3)
X_alinha_teste = X_alinha_teste.reshape(-1, 90000, 3)

red_alinha_treino = X_alinha_treino[:,:,0]
blue_alinha_treino = X_alinha_treino[:,:,1]
green_alinha_treino = X_alinha_treino[:,:,2]

red_alinha_teste = X_alinha_teste[:,:,0]
blue_alinha_teste = X_alinha_teste[:,:,1]
green_alinha_teste = X_alinha_teste[:,:,2]

red_alinha_treino.shape

(1074, 90000)

# Treinar modelo com imagens redimensionadas

In [6]:
model = criar_modelo()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit([red_treino, green_treino, blue_treino], y_treino, epochs=10, batch_size=32)

score = model.evaluate([red_teste, green_teste, blue_teste], y_teste, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1] * 100, "%")

Epoch 1/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 100ms/step - accuracy: 0.5719 - loss: 2.1783
Epoch 2/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.7375 - loss: 0.8222
Epoch 3/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - accuracy: 0.8176 - loss: 0.4377
Epoch 4/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - accuracy: 0.8367 - loss: 0.4065
Epoch 5/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 104ms/step - accuracy: 0.7937 - loss: 0.4665
Epoch 6/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.8544 - loss: 0.3452
Epoch 7/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - accuracy: 0.8355 - loss: 0.3623
Epoch 8/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - accuracy: 0.8326 - loss: 0.3583
Epoch 9/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - accuracy: 0.8261 - loss: 0.4721
Epoch 10/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - accuracy: 0.8175 - loss: 0.4287
Test loss: 0.41149675846099854
Test accuracy: 84.26443338394165 %


# Treinar modelo com imagens redimensionadas e rotacionadas

In [7]:
model = criar_modelo()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit([red_alinha_treino, green_alinha_treino, blue_alinha_treino], y_treino, epochs=10, batch_size=32)

score = model.evaluate([red_alinha_teste, green_alinha_teste, blue_alinha_teste], y_teste, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1] * 100, "%")


Epoch 1/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 97ms/step - accuracy: 0.5618 - loss: 572.0127
Epoch 2/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - accuracy: 0.8006 - loss: 25.9102
Epoch 3/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 95ms/step - accuracy: 0.7307 - loss: 33.7388
Epoch 4/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 96ms/step - accuracy: 0.7843 - loss: 18.8342
Epoch 5/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - accuracy: 0.7652 - loss: 19.8045
Epoch 6/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - accuracy: 0.7804 - loss: 24.7991
Epoch 7/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - accuracy: 0.8109 - loss: 19.7437
Epoch 8/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 95ms/step - accuracy: 0.8485 - loss: 6.6731
Epoch 9/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - accuracy: 0.8062 - loss: 9.0765
Epoch 10/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - accuracy: 0.8169 - loss: 12.3948
Test loss: 12.102677345275879
Test accuracy: 84.63687300682068 %
